## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [71]:
from lib.data import Data, get_buy_and_hold_df, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
    plot_avg_portfolio_allocation, 
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [5]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

In [31]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
n_assets = 8
assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# number of principal components (None means no PCA)
pca_ncp = None

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"
]

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV']

#### Instantiate `Data` object

In [32]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp)
data

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['NOV', 'XOM', 'VZ', 'MSFT', 'AAP', 'AMAT', 'MCD', 'NKE'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], pca_ncp=None)

In [59]:
# first date of trading = minimum date in second batch
start_trading_date = list(data.batches)[1][1].index.min()

In [60]:
buy_and_hold = get_buy_and_hold_df(data.df.loc[data.df.index >= start_trading_date, :])
buy_and_hold

,returns,cumulative_returns
Date,,
2010-05-26,-0.011442,0.988558
2010-05-27,0.032069,1.020261
2010-05-28,-0.012892,1.007108
2010-06-01,-0.026916,0.980001
2010-06-02,0.023953,1.003475
...,...,...
2017-12-21,0.007253,2.610505
2017-12-22,-0.002004,2.605274
2017-12-26,0.000503,2.606585


### `RRL` - Training and validation

In [36]:
initial_invest = 100
n_epochs = 50

#### No transaction fees 

$\delta=0$

In [41]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_no_fees_5ncp.pkl"
file_path = "./backup/no_pca/rrl_no_fees_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  26%|██▌       | 13/50 [00:00<00:01, 20.76it/s, sharpe_ratio=0.0112]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.6890234150892693
Cumulative profits: 117.27539847128003
Training window: 2010-06-30/2010-10-14


Training in progress...:  74%|███████▍  | 37/50 [00:01<00:00, 21.50it/s, sharpe_ratio=0.23] 


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.1607521830275096
Cumulative profits: 122.68398584456024
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.8493854506103535
Cumulative profits: 223.6653761736542
Training window: 2017-03-31/2017-07-18


Training in progress...:  66%|██████▌   | 33/50 [00:01<00:00, 26.33it/s, sharpe_ratio=-.0892]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.759961081843174
Cumulative profits: 233.66253957846627
Training window: 2017-08-23/2017-12-07


Training in progress...:  74%|███████▍  | 37/50 [00:01<00:00, 26.27it/s, sharpe_ratio=0.075] 


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [42]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_10bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_10bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  30%|███       | 15/50 [00:00<00:01, 18.84it/s, sharpe_ratio=0.014] 


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.6134673454408435
Cumulative profits: 117.53349396344612
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 25.49it/s, sharpe_ratio=0.218]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.2054330356092628
Cumulative profits: 123.13022089704619
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -2.348992702046149
Cumulative profits: 160.21555415271348
Training window: 2017-03-31/2017-07-18


Training in progress...:  78%|███████▊  | 39/50 [00:01<00:00, 22.68it/s, sharpe_ratio=-.111]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.6408082124186375
Cumulative profits: 162.90264375186777
Training window: 2017-08-23/2017-12-07


Training in progress...:  64%|██████▍   | 32/50 [00:01<00:00, 26.36it/s, sharpe_ratio=0.0299]


Less than 30 observations in the validation set after preprocessing.


In [43]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_30bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  22%|██▏       | 11/50 [00:00<00:01, 19.84it/s, sharpe_ratio=-.00234]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.319852850906812
Cumulative profits: 110.518085486617
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 28.07it/s, sharpe_ratio=0.16] 


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -0.5224420957518631
Cumulative profits: 107.9356315974264
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.327499666153272
Cumulative profits: 109.49084199859155
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 25.50it/s, sharpe_ratio=-.185]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.6704332970736927
Cumulative profits: 111.43045403378451
Training window: 2017-08-23/2017-12-07


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 29.90it/s, sharpe_ratio=0.0514]


Less than 30 observations in the validation set after preprocessing.


In [44]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_50bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  26%|██▌       | 13/50 [00:00<00:01, 23.38it/s, sharpe_ratio=-.0296]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.3456498934807946
Cumulative profits: 106.0318996816068
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 26.60it/s, sharpe_ratio=0.107] 


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -1.1313693695889313
Cumulative profits: 100.64814839319284
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -5.072498609280853
Cumulative profits: 41.09079228366252
Training window: 2017-03-31/2017-07-18


Training in progress...:   4%|▍         | 2/50 [00:00<00:03, 15.96it/s, sharpe_ratio=-.259]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.4083169664673647
Cumulative profits: 42.68629080875883
Training window: 2017-08-23/2017-12-07


Training in progress...:   2%|▏         | 1/50 [00:00<00:03, 12.84it/s, sharpe_ratio=0.0887]


Less than 30 observations in the validation set after preprocessing.


### Backtest

In [45]:
version = 1
file_names = [f"rrl_{fees}_{version}" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/no_pca/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [46]:
dfs_backtest = {
    k: create_backtest_dataset(v)
    for k, v in results.items()
}

#### Cumulative profits

In [49]:
initial_invest = 100

In [61]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [62]:
setting = "rrl_no_fees_1"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [63]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [64]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [72]:
layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)

fig = plot_avg_portfolio_allocation(results[setting], layout)

fig.show()